In [9]:
import os
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import cv2 
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report , confusion_matrix

from tqdm import tqdm
from keras.layers import Conv2D, MaxPooling2D , BatchNormalization ,Dropout ,Flatten , Dense , Input
from tensorflow.keras.applications import VGG16
from keras.models import Sequential

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
plt.style.use('ggplot')

dataDir = 'datasets\Potato'
selectedClasses = ['Potato___healthy',
                   'Potato___Early_blight',
                   'Potato___Late_blight',
                   ]

imgPaths = []
labels = []
for className in os.listdir(dataDir):
    if className in selectedClasses :                  # Select you classes above
        classPath = os.path.join(dataDir,className)
        for img in os.listdir(classPath):
            imgPath = os.path.join(classPath,img)
            imgPaths.append(imgPath)
            labels.append(className)
            
# Convert the 2 lists to dataframe to easy use 
df = pd.DataFrame({
    'imgPath':imgPaths,
    'label':labels
})

df = df.sample(frac=1).reset_index(drop=True)            # Shuffle

df['label'] = df['label'].replace({'Potato___healthy':0,
                                    'Potato___Early_blight':1,
                                    'Potato___Late_blight':2,
                                    }).astype(int)

IMG_SIZE = (150,150)   # to free some resources and reduce the execution time
imgs = []
for imgPath in tqdm(df['imgPath'], total=len(df)):
    img = cv2.imread(imgPath)
    img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE)
    imgs.append(img)

# convert them to numpy array to we can split them 
images = np.array(imgs)
labels = np.array(df['label'])

images = images / 255.0         # normalize from 0 --> 255 to 0 --> 1  to reduce the execution time 

# Spliting 
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42 ,shuffle=True)

print(f'Shape of X_train : {X_train.shape}')
print(f'Shape of X_test : {X_test.shape}')
print(f'Shape of y_train : {y_train.shape}')
print(f'Shape of y_test : {y_test.shape}')

custom_input = Input(shape=(150, 150, 3))           # To customize the input shape of PreTrained model  

with tf.device('/GPU:0'):          # to use GPU
    Model = Sequential([

        VGG16(weights='imagenet', include_top=False, input_tensor=custom_input),
        
        Flatten(),                                     # because we ignore the flatten and dense layers when include_top = False 
        
        Dense(128,activation='relu'),
        
        Dropout(0.2),

        Dense(3 ,activation='softmax')
    ])

VGG16Layers = Model.layers[0]
for layer in VGG16Layers.layers[1:-3]:           # freez all layers except the first and last 3 layers, we will make them trainable (weghts changes with training)
    layer.trainable = False

Model.compile(optimizer='adam',loss='sparse_categorical_crossentropy' ,metrics=['accuracy'])

history = Model.fit(X_train,y_train,
                         validation_split = 0.2 , #validation_data=(X_test,y_test),
                         epochs=10,
                         batch_size=100, 
                         verbose=1,
                         callbacks=[tf.keras.callbacks.EarlyStopping(
                                            patience=5,
                                            monitor='val_accuracy',
                                            restore_best_weights=True)])

y_pred = Model.predict(X_test)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

test_result = [np.argmax(x) for x in y_pred]

CM = confusion_matrix(y_test, test_result)
sns.heatmap(CM, center = True,cmap='summer',annot=True ,fmt='.5g')
plt.show()

ClassificationReport = classification_report(y_test,test_result)
print('Classification Report is : \n', ClassificationReport )

Model.save('Potato.keras')

C:\Users\sarth\AppData\Local\Temp\ipykernel_18872\3288513875.py:48: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['label'].replace({'Potato___healthy':0,
100%|██████████| 2152/2152 [00:02<00:00, 804.91it/s]


Shape of X_train : (1721, 150, 150, 3)
Shape of X_test : (431, 150, 150, 3)
Shape of y_train : (1721,)
Shape of y_test : (431,)
Epoch 1/10


c:\Users\sarth\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_203']. Received: the structure of inputs=*
  warnings.warn(


14/14 ━━━━━━━━━━━━━━━━━━━━ 60s 4s/step - accuracy: 0.4530 - loss: 1.9344 - val_accuracy: 0.6435 - val_loss: 0.7608
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.7194 - loss: 0.6161 - val_accuracy: 0.8261 - val_loss: 0.4904
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.8951 - loss: 0.3138 - val_accuracy: 0.8870 - val_loss: 0.2837
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.8983 - loss: 0.2688 - val_accuracy: 0.8841 - val_loss: 0.3042
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 57s 4s/step - accuracy: 0.9255 - loss: 0.1673 - val_accuracy: 0.8957 - val_loss: 0.3212
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.9501 - loss: 0.1402 - val_accuracy: 0.9362 - val_loss: 0.1993
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 57s 4s/step - accuracy: 0.9626 - loss: 0.0913 - val_accuracy: 0.9304 - val_loss: 0.1860
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.9696 - loss: 0.0799 - val_accuracy: 0.9420 - val_loss: 0.1596
Epo

In [10]:
def predictDisease(imgPath,crop_name,crop_array):
    test_df = pd.DataFrame({
        'imgPath':imgPath,
    }, index=[0])

    IMG_SIZE = (150,150)   # to free some resources and reduce the execution time
    imgs = []
    for imgPath in tqdm(test_df['imgPath'], total=len(test_df)):
        img = cv2.imread(imgPath)
        img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, IMG_SIZE)
        imgs.append(img)

    images = np.array(imgs)
    images = images / 255.0         # normalize from 0 --> 255 to 0 --> 1  to reduce the execution time 

    # new_model = tf.keras.models.load_model('pepper_bell_model.keras')
    print(f'{crop_name}.keras')
    new_model = tf.keras.models.load_model(f'{crop_name}.keras')

    test_result = new_model.predict(images)
    print(test_result)
    test_result = [np.argmax(x) for x in test_result]
    print(test_result)

    print(crop_array[test_result[0]])

corn = ["Corn__healthy","Corn__blight", "Corn__common_rust","Corn__gray_leaf_spot"]
pepper_bell = ["Pepper__bell___healthy", "Pepper__bell___Bacterial_spot"]
potato = ["Potato___healthy", "Potato___Early_blight","Potato___Late_blight"]
tomato = ["Tomato_healthy", "Tomato__Target_Spot","Tomato__Tomato_mosaic_virus","Tomato__Tomato_YellowLeaf__Curl_Virus","Tomato_Bacterial_spot"]

# predictDisease("demo\Corn__gray_leaf_spot_1.JPG", "Corn", corn)
# predictDisease("demo\Pepper__bell___Bacterial_spot_3.JPG", "Pepper Bell", pepper_bell)
predictDisease("demo\Potato___Early_blight_3.JPG", "potato", potato)
# predictDisease("demo\Tomato_Bacterial_spot_3.JPG", "Tomato", tomato)


100%|██████████| 1/1 [00:00<00:00, 65.38it/s]


potato.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
[[1.4737602e-16 1.0000000e+00 1.8552462e-11]]
[1]
Potato___Early_blight
